
# Week 2 Lab - Calculating Probability and Proportion

## Justin Parsons
Week 2 MSDS 650 8w2

In [1]:
import pandas as pd
from scipy import stats

First, we load the data and take a look at it. The data was also opened in the Jupyter-Lab editor to avoid munging the data.

In [2]:
#Since I'm limited for time this week I chose to use the cleaned data.
flights_df = pd.read_csv('assign_wk2/flights_clean.csv')

In [3]:
flights_df.sample(5, random_state=42)

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
1030241,2015,3,11,3,B6,305,EWR,FLL,1610,1608.0,-2.0,189.0,174.0,1919,1902.0,-17.0,0,0
1766167,2015,4,26,7,OO,5248,IAH,IND,1335,1335.0,0.0,137.0,122.0,1652,1637.0,-15.0,0,0
332334,2015,1,23,5,B6,1289,EWR,SJU,730,729.0,-1.0,233.0,217.0,1223,1206.0,-17.0,0,0
3809702,2015,8,29,6,WN,4451,MDW,DEN,1335,1331.0,-4.0,155.0,129.0,1510,1440.0,-30.0,0,0
4675064,2015,11,24,2,HA,24,OGG,OAK,1225,1257.0,32.0,295.0,275.0,1920,1932.0,12.0,0,0


In [4]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5245484 entries, 0 to 5245483
Data columns (total 18 columns):
 #   Column               Dtype  
---  ------               -----  
 0   year                 int64  
 1   month                int64  
 2   day                  int64  
 3   day_of_week          int64  
 4   airline              object 
 5   flight_number        int64  
 6   origin_airport       object 
 7   destination_airport  object 
 8   scheduled_departure  int64  
 9   departure_time       float64
 10  departure_delay      float64
 11  scheduled_time       float64
 12  elapsed_time         float64
 13  scheduled_arrival    int64  
 14  arrival_time         float64
 15  arrival_delay        float64
 16  diverted             int64  
 17  cancelled            int64  
dtypes: float64(6), int64(9), object(3)
memory usage: 720.4+ MB


In [5]:
flights_df.shape

(5245484, 18)

So, we see that there are over 5x10^6 samples in this dataset. Also, it is a mix of int, float, and strings which may need to be re-mapped to different types at some point. Our first goal is to determine which airports had the largest proportion of late arrivals and also which had the smallest proportion. Note, the question is which airports had the lowest (highest) proportion at their destination. This means we are concerned with the origin airport and the arrival delay. 

In [6]:
flights_df.groupby('origin_airport').size()

origin_airport
ABE     2235
ABI     2232
ABQ    18980
ABR      663
ABY      867
       ...  
WRG      653
WYS      208
XNA     8987
YAK      657
YUM     1854
Length: 322, dtype: int64

In [7]:
flights_df

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
0,2015,1,1,4,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,430,408.0,-22.0,0,0
1,2015,1,1,4,AA,2336,LAX,PBI,10,2.0,-8.0,280.0,279.0,750,741.0,-9.0,0,0
2,2015,1,1,4,US,840,SFO,CLT,20,18.0,-2.0,286.0,293.0,806,811.0,5.0,0,0
3,2015,1,1,4,AA,258,LAX,MIA,20,15.0,-5.0,285.0,281.0,805,756.0,-9.0,0,0
4,2015,1,1,4,AS,135,SEA,ANC,25,24.0,-1.0,235.0,215.0,320,259.0,-21.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245479,2015,12,31,4,B6,688,LAX,BOS,2359,2355.0,-4.0,320.0,298.0,819,753.0,-26.0,0,0
5245480,2015,12,31,4,B6,745,JFK,PSE,2359,2355.0,-4.0,227.0,215.0,446,430.0,-16.0,0,0
5245481,2015,12,31,4,B6,1503,JFK,SJU,2359,2350.0,-9.0,221.0,222.0,440,432.0,-8.0,0,0
5245482,2015,12,31,4,B6,333,MCO,SJU,2359,2353.0,-6.0,161.0,157.0,340,330.0,-10.0,0,0


We will use apply to create a column of bools where 1 indicates a delay and 0 indicates no delay. From there we should be able to calculate the proportion of late arrivals per airport. 

In [8]:
#Here I'm inserting a calculation for a total flights column
flights_df['total'] = flights_df.groupby('origin_airport')['arrival_delay'].size()
flights_df

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,total
0,2015,1,1,4,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,430,408.0,-22.0,0,0,NaN
1,2015,1,1,4,AA,2336,LAX,PBI,10,2.0,-8.0,280.0,279.0,750,741.0,-9.0,0,0,NaN
2,2015,1,1,4,US,840,SFO,CLT,20,18.0,-2.0,286.0,293.0,806,811.0,5.0,0,0,NaN
3,2015,1,1,4,AA,258,LAX,MIA,20,15.0,-5.0,285.0,281.0,805,756.0,-9.0,0,0,NaN
4,2015,1,1,4,AS,135,SEA,ANC,25,24.0,-1.0,235.0,215.0,320,259.0,-21.0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245479,2015,12,31,4,B6,688,LAX,BOS,2359,2355.0,-4.0,320.0,298.0,819,753.0,-26.0,0,0,NaN
5245480,2015,12,31,4,B6,745,JFK,PSE,2359,2355.0,-4.0,227.0,215.0,446,430.0,-16.0,0,0,NaN
5245481,2015,12,31,4,B6,1503,JFK,SJU,2359,2350.0,-9.0,221.0,222.0,440,432.0,-8.0,0,0,NaN
5245482,2015,12,31,4,B6,333,MCO,SJU,2359,2353.0,-6.0,161.0,157.0,340,330.0,-10.0,0,0,NaN


In [9]:
flights_df['late_arrival'] = flights_df.arrival_delay.apply(lambda x: x < 0)

In [10]:
flights_df

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,total,late_arrival
0,2015,1,1,4,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,430,408.0,-22.0,0,0,NaN,True
1,2015,1,1,4,AA,2336,LAX,PBI,10,2.0,-8.0,280.0,279.0,750,741.0,-9.0,0,0,NaN,True
2,2015,1,1,4,US,840,SFO,CLT,20,18.0,-2.0,286.0,293.0,806,811.0,5.0,0,0,NaN,False
3,2015,1,1,4,AA,258,LAX,MIA,20,15.0,-5.0,285.0,281.0,805,756.0,-9.0,0,0,NaN,True
4,2015,1,1,4,AS,135,SEA,ANC,25,24.0,-1.0,235.0,215.0,320,259.0,-21.0,0,0,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245479,2015,12,31,4,B6,688,LAX,BOS,2359,2355.0,-4.0,320.0,298.0,819,753.0,-26.0,0,0,NaN,True
5245480,2015,12,31,4,B6,745,JFK,PSE,2359,2355.0,-4.0,227.0,215.0,446,430.0,-16.0,0,0,NaN,True
5245481,2015,12,31,4,B6,1503,JFK,SJU,2359,2350.0,-9.0,221.0,222.0,440,432.0,-8.0,0,0,NaN,True
5245482,2015,12,31,4,B6,333,MCO,SJU,2359,2353.0,-6.0,161.0,157.0,340,330.0,-10.0,0,0,NaN,True


In [11]:
late_df = flights_df.groupby(['origin_airport','late_arrival']).size().unstack().reset_index()
late_df

late_arrival,origin_airport,False,True
0,ABE,870,1365
1,ABI,727,1505
2,ABQ,7429,11551
3,ABR,273,390
4,ABY,328,539
...,...,...,...
317,WRG,252,401
318,WYS,66,142
319,XNA,3927,5060
320,YAK,178,479


We need another column that is the proportion of late (true) to total flights. We'll add this column to the late_df dataframe.

In [12]:
late_df.groupby('origin_airport')
late_df

late_arrival,origin_airport,False,True
0,ABE,870,1365
1,ABI,727,1505
2,ABQ,7429,11551
3,ABR,273,390
4,ABY,328,539
...,...,...,...
317,WRG,252,401
318,WYS,66,142
319,XNA,3927,5060
320,YAK,178,479


In [13]:
late_df['total_flights'] = late_df.False + late_df.True

SyntaxError: invalid syntax (3149992312.py, line 1)